In [1]:
import numpy as np
from numpy import array
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2
from sympy.matrices import Matrix

In [2]:
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta - joint angles
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') #link offsets
a0, a1, a2, a3, a4, a5, a6 = symbols('a1:8') #link lengths
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7') #Twist

In [3]:
#DH Parameters
# KUKA KR210

DH_Params = {alpha0:      0.0,  a0:       0.0,  d1:     0.75, q1:         q1,
             alpha1:    -pi/2,  a1:      0.35,  d2:      0.0, q2:    q2-pi/2,
             alpha2:      0.0,  a2:      1.25,  d3:      0.0, q3:         q3,
             alpha3:    -pi/2,  a3:    -0.054,  d4:     1.50, q4:         q4,
             alpha4:     pi/2,  a4:       0.0,  d5:      0.0, q5:         q5,
             alpha5:    -pi/2,  a5:       0.0,  d6:      0.0, q6:         q6,
             alpha6:      0.0,  a6:       0.0,  d7:    0.303, q7:        0.0,}

In [4]:
def TF_Matrix(alpha, a, d, q):
    TF =    Matrix([[            cos(q),           -sin(q),           0,             a],
                    [ sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                    [sin(q)* sin(alpha), cos(q)*sin(alpha),  cos(alpha),  cos(alpha)*d],
                    [                 0,                 0,           0,             1]])
    return TF

In [5]:
#Homogenous Transformations

T0_1 = TF_Matrix(alpha0, a0, d1, q1).subs(DH_Params)
T1_2 = TF_Matrix(alpha1, a1, d2, q2).subs(DH_Params)
T2_3 = TF_Matrix(alpha2, a2, d3, q3).subs(DH_Params)
T3_4 = TF_Matrix(alpha3, a3, d4, q4).subs(DH_Params)
T4_5 = TF_Matrix(alpha4, a4, d5, q5).subs(DH_Params)
T5_6 = TF_Matrix(alpha5, a5, d6, q6).subs(DH_Params)
T6_G = TF_Matrix(alpha6, a6, d7, q7).subs(DH_Params)

In [6]:
# Correction Needed to account of orientaion difference between definition of gripper link in URDF versus DH convention
R_z = Matrix([[ cos(np.pi), -sin(np.pi),  0.0,  0.0],
              [ sin(np.pi),  cos(np.pi),  0.0,  0.0],
              [        0.0,         0.0,  1.0,  0.0],
              [        0.0,         0.0,  0.0,  1.0]])
R_y = Matrix([[  cos(-np.pi/2),   0.0,    sin(-np.pi/2),  0.0],
              [         0.0,   1.0,           0.0,  0.0],
              [ -sin(-np.pi/2),   0.0,   cos(-np.pi/2),    0.0],
              [         0.0,   0.0,        0.0,     1.0]])

In [7]:
T0_1

Matrix([
[cos(q1), -sin(q1), 0,    0],
[sin(q1),  cos(q1), 0,    0],
[      0,        0, 1, 0.75],
[      0,        0, 0,    1]])

In [8]:
T1_2

Matrix([
[sin(q2),  cos(q2), 0, 0.35],
[      0,        0, 1,    0],
[cos(q2), -sin(q2), 0,    0],
[      0,        0, 0,    1]])

In [9]:
T2_3

Matrix([
[cos(q3), -sin(q3), 0, 1.25],
[sin(q3),  cos(q3), 0,    0],
[      0,        0, 1,    0],
[      0,        0, 0,    1]])

In [10]:
T3_4

Matrix([
[ cos(q4), -sin(q4), 0, -0.054],
[       0,        0, 1,    1.5],
[-sin(q4), -cos(q4), 0,      0],
[       0,        0, 0,      1]])

In [11]:
T4_5

Matrix([
[cos(q5), -sin(q5),  0, 0],
[      0,        0, -1, 0],
[sin(q5),  cos(q5),  0, 0],
[      0,        0,  0, 1]])

In [12]:
T5_6

Matrix([
[ cos(q6), -sin(q6), 0, 0],
[       0,        0, 1, 0],
[-sin(q6), -cos(q6), 0, 0],
[       0,        0, 0, 1]])

In [13]:
T6_G

Matrix([
[1, 0, 0,     0],
[0, 1, 0,     0],
[0, 0, 1, 0.303],
[0, 0, 0,     1]])

In [14]:
R_z

Matrix([
[                -1.0, -1.22464679914735e-16,   0,   0],
[1.22464679914735e-16,                  -1.0,   0,   0],
[                   0,                     0, 1.0,   0],
[                   0,                     0,   0, 1.0]])

In [15]:
R_y

Matrix([
[6.12323399573677e-17,   0,                 -1.0,   0],
[                   0, 1.0,                    0,   0],
[                 1.0,   0, 6.12323399573677e-17,   0],
[                   0,   0,                    0, 1.0]])

In [16]:
R_corr = simplify(R_z * R_y)
R_corr

Matrix([
[-6.12323399573677e-17, -1.22464679914735e-16,                   1.0,   0],
[ 7.49879891330929e-33,                  -1.0, -1.22464679914735e-16,   0],
[                  1.0,                     0,  6.12323399573677e-17,   0],
[                    0,                     0,                     0, 1.0]])

In [17]:
T0_G = simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_G)

In [18]:
T0_G

Matrix([
[((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*cos(q6) - (-sin(q1)*cos(q4) + sin(q4)*sin(q2 + q3)*cos(q1))*sin(q6), -((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*sin(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*cos(q6), -(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3), -0.303*sin(q1)*sin(q4)*sin(q5) + 1.25*sin(q2)*cos(q1) - 0.303*sin(q5)*sin(q2 + q3)*cos(q1)*cos(q4) - 0.054*sin(q2 + q3)*cos(q1) + 0.303*cos(q1)*cos(q5)*cos(q2 + q3) + 1.5*cos(q1)*cos(q2 + q3) + 0.35*cos(q1)],
[ ((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*sin(q6), -((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*sin(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*cos(q6), -(sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*sin(

In [19]:
T_total = simplify(T0_G * R_corr)

In [20]:
Origin_Pose = {q1:0, q2:0, q3:0, q4:0, q5:0, q6:0, q7:0}
Origin_Matrix = T_total.subs(Origin_Pose)
Origin_Matrix

Matrix([
[                  1.0,                     0, 6.12323399573677e-17, 2.153],
[-7.49879891330929e-33,                   1.0, 1.22464679914735e-16,     0],
[-6.12323399573677e-17, -1.22464679914735e-16,                  1.0, 1.946],
[                    0,                     0,                    0,   1.0]])

In [21]:
Origin_Pose = {q1:0, q2:0, q3:0, q4:0, q5:0, q6:0, q7:0}
Origin_Matrix = T0_G.subs(Origin_Pose)
Origin_Matrix

Matrix([
[0,  0, 1, 2.153],
[0, -1, 0,     0],
[1,  0, 0, 1.946],
[0,  0, 0,     1]])

In [22]:
Origin_Pose = {q1:1.35, q2:1.09, q3:-0.09, q4:3.18, q5:0.08, q6:0.38, q7:0}
Origin_Matrix = T_total.subs(Origin_Pose)
Origin_Matrix

Matrix([
[   0.1356618670489,  0.820447998868102, -0.555392598962332,  0.528022602556079],
[ 0.590392455018793, -0.517118815799423, -0.619697409550084,   2.34821238060766],
[-0.795633462648278, -0.243830292415708, -0.554539612304461, -0.204353032317026],
[                 0,                  0,                  0,                1.0]])